In [1]:
import warnings
warnings.filterwarnings('ignore')
import pickle
import numpy as np
import pandas as pd
import json
from textblob import TextBlob
import nltk
nltk.download('punkt')
from scipy import spatial
import torch
import spacy
en_nlp = spacy.load('en')
from models import InferSent

[nltk_data] Downloading package punkt to /Users/u403041/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Read json and transform to csv

## Create dictionary of sentence embeddings 

In [2]:
df = pd.read_csv('train.csv')

In [3]:
paras = df["context"].unique().tolist()
print(len(paras))
paras[1]

18849


'Following the disbandment of Destiny\'s Child in June 2005, she released her second solo album, B\'Day (2006), which contained hits "Déjà Vu", "Irreplaceable", and "Beautiful Liar". Beyoncé also ventured into acting, with a Golden Globe-nominated performance in Dreamgirls (2006), and starring roles in The Pink Panther (2006) and Obsessed (2009). Her marriage to rapper Jay Z and portrayal of Etta James in Cadillac Records (2008) influenced her third album, I Am... Sasha Fierce (2008), which saw the birth of her alter-ego Sasha Fierce and earned a record-setting six Grammy Awards in 2010, including Song of the Year for "Single Ladies (Put a Ring on It)". Beyoncé took a hiatus from music in 2010 and took over management of her career; her fourth album 4 (2011) was subsequently mellower in tone, exploring 1970s funk, 1980s pop, and 1990s soul. Her critically acclaimed fifth studio album, Beyoncé (2013), was distinguished from previous releases by its experimental production and exploratio

In [4]:
blob = TextBlob(" ".join(paras))
sentences = [item.raw for item in blob.sentences]
print(len(sentences))

92530


In [5]:
# Load our pre-trained model 
from models import InferSent
V = 2
MODEL_PATH = 'infersent%s.pkl' % V
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': V}
infersent = InferSent(params_model)
infersent.load_state_dict(torch.load(MODEL_PATH))

<All keys matched successfully>

In [6]:
# Set word vector path for the model
W2V_PATH = 'crawl-300d-2M.vec'
infersent.set_w2v_path(W2V_PATH)

In [7]:
# Build the vocabulary of word vectors
infersent.build_vocab(sentences, tokenize=True)

Found 89159(/109620) words with w2v vectors
Vocab size : 89159


In [9]:
import pickle
with open('infersent.pkl', 'wb') as f:
    pickle.dump(infersent, f)

In [11]:
dict_embeddings = {}

In [12]:
# Encode your sentences
# dimension 4096
for i in range(len(sentences)):
    try:
        dict_embeddings[sentences[i]] = infersent.encode([sentences[i]], tokenize=True)
    except:
        print(sentences[i])
        print(i)
print(len(dict_embeddings))

92379


In [16]:
# Encode your questions
questions = df["question"].tolist()
print(len(questions))
for i in range(len(questions)):
    try:
        dict_embeddings[questions[i]] = infersent.encode([questions[i]], tokenize=True)
    except:
        print(questions[i])
        print(i)
print(len(dict_embeddings))

84600
176931


In [17]:
with open('dict_embeddings2.pickle', 'wb') as handle:
    pickle.dump(dict_embeddings, handle)